## extracting from recon3d

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [2]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-30


In [3]:
_REQUIRED_REACTION_ATTRIBUTES = [
    "id",
    "name",
    "metabolites",
    "lower_bound",
    "upper_bound",
#     "gene_reaction_rule",
]

_REQUIRED_METABOLITE_ATTRIBUTES = ["id", "name", "charge","formula", "compartment"]
_ORDERED_OPTIONAL_METABOLITE_KEYS = [
   
    "_bound",
    "notes",
    "annotation",
]


In [4]:
# def metabolite_to_dict(metabolite):
#     new_met = dict()
#     for key in _REQUIRED_METABOLITE_ATTRIBUTES:
#         new_met[key] = _fix_type(getattr(metabolite, key))
#     return new_met
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [5]:
reaction_list = []
reaction_list = ['HEX1',
                'PGI',
                'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                # 'PEPtm',
                'PEPCKm',
                'PYK',
                'PCm',
                'LDH_L',
                'CO2tm',
                # 'Htm',
                'O2tm',
                 'H2Otm',
                 
#                 'G6Pter',
#                 'G6PPer',
#                 'GLCter',
#                 'GLCt1',
                'PYRt2m', 
#                 'H2Oter', 
#                 'PIter', 
#                 'H2Ot', 
#                 'PIt', 
#                 'Ht', 
#                 'L_LACt2r', #lactose transport between lactate in cytosol and extracellular 
#                  'PYRt2',
                'ADK1',
                'ATPM',
#                 'DM_nadh'
                ]

In [6]:
metabolite_list=[]
for reaction in reaction_list:
    rxn = R3D.reactions.get_by_id(reaction)
    for met in rxn.metabolites:
        if met.id not in metabolite_list:
            metabolite_list.append(met.id)
len(metabolite_list)

35

In [7]:
met_df=pd.DataFrame()
for met in metabolite_list:
    r3d_met= R3D.metabolites.get_by_id(met)
    m=metabolite_to_dict(r3d_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=met_df.append(df_2)

met_df=met_df.set_index('id')
met_df

,name,charge,formula,compartment
id,,,,
adp_c,ADP C10H12N5O10P2,-3,C10H12N5O10P2,c
atp_c,ATP C10H12N5O13P3,-4,C10H12N5O13P3,c
g6p_c,D-Glucose 6-phosphate,-2,C6H11O9P,c
glc__D_c,D-Glucose,0,C6H12O6,c
h_c,H+,0,H,c
f6p_c,D-Fructose 6-phosphate,-2,C6H11O9P,c
fdp_c,"D-Fructose 1,6-bisphosphate",-4,C6H10O12P2,c
h2o_c,H2O H2O,0,H2O,c
pi_c,Phosphate,0,HO4P,c


In [8]:
csv_met = os.path.join(data_dir,"gly_gluco_met_df.csv")
met_df.to_csv(csv_met)

In [9]:
rxn_df=pd.DataFrame()
for rxn in reaction_list:
    r3d_rxn= R3D.reactions.get_by_id(rxn)
    r=reaction_to_dict(r3d_rxn)
#     print(r)
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df=rxn_df.append(df)

rxn_df=rxn_df.set_index('id')
rxn_df

,name,metabolites,lower_bound,upper_bound,gene_reaction_rule,subsystem,notes,annotation
id,,,,,,,,
HEX1,Hexokinase (D-glucose:ATP),"{'adp_c': 1.0, 'atp_c': -1.0, 'g6p_c': 1.0, 'g...",0.0,1000.0,3098_AT3 or 80201_AT1 or 2645_AT3 or 3098_AT4 ...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['HEX1']},"{'bigg.reaction': ['HEX1'], 'biocyc': ['META:G..."
PGI,Glucose-6-phosphate isomerase,"{'f6p_c': 1.0, 'g6p_c': -1.0}",-1000.0,1000.0,2821_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGI']},"{'bigg.reaction': ['PGI'], 'biocyc': ['META:PG..."
FBP,Fructose-bisphosphatase,"{'f6p_c': 1.0, 'fdp_c': -1.0, 'h2o_c': -1.0, '...",0.0,1000.0,2203_AT1 or 8789_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['FBP']},"{'bigg.reaction': ['FBP'], 'ec-code': ['3.1.3...."
PFK,Phosphofructokinase,"{'adp_c': 1.0, 'atp_c': -1.0, 'f6p_c': -1.0, '...",0.0,1000.0,5214_AT1 or 5211_AT2 or (5214_AT1 and 5211_AT1...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PFK']},"{'bigg.reaction': ['PFK'], 'ec-code': ['2.7.1...."
FBA,Fructose-bisphosphate aldolase,"{'dhap_c': 1.0, 'fdp_c': -1.0, 'g3p_c': 1.0}",-1000.0,1000.0,226_AT2 or 226_AT1 or 226_AT3 or 230_AT1 or 22...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['FBA']},"{'bigg.reaction': ['FBA'], 'ec-code': ['4.1.2...."
TPI,Triose-phosphate isomerase,"{'dhap_c': -1.0, 'g3p_c': 1.0}",-1000.0,1000.0,7167_AT1 or 286016_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['TPI']},"{'bigg.reaction': ['TPI'], 'biocyc': ['META:TR..."
GAPD,Glyceraldehyde-3-phosphate dehydrogenase,"{'13dpg_c': 1.0, 'g3p_c': -1.0, 'h_c': 1.0, 'n...",-1000.0,1000.0,2597_AT1 or 26330_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['GAPD']},"{'bigg.reaction': ['GAPD'], 'biocyc': ['META:G..."
PGK,Phosphoglycerate kinase,"{'13dpg_c': 1.0, '3pg_c': -1.0, 'adp_c': 1.0, ...",-1000.0,1000.0,5230_AT1 or 375056_AT1 or 5232_AT1,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGK']},"{'bigg.reaction': ['PGK'], 'biocyc': ['META:PH..."
PGM,Phosphoglycerate mutase,"{'2pg_c': -1.0, '3pg_c': 1.0}",-1000.0,1000.0,669_AT1 or 5223_AT1 or 5224_AT2 or 5224_AT1 or...,Glycolysis/gluconeogenesis,{'original_bigg_ids': ['PGM']},"{'bigg.reaction': ['PGM'], 'biocyc': ['META:3P..."


In [10]:
csv_rxn = os.path.join(data_dir,"gly_gluco_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [11]:
json_rxn = os.path.join(data_dir,"gly_gluco_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)
    # orient="columns"
    